<img align="right" src="https://raw.githubusercontent.com/GT4SD/gt4sd-core/main/docs/_static/gt4sd_logo.png" alt="logo" width="100" height="100"/>

##  Demo: Regression Transformer in the Generative Toolkit for Scientific Discovery


In [ ]:
import logging, sys
logging.disable(sys.maxsize)


from gt4sd.algorithms.conditional_generation.regression_transformer import (
    RegressionTransformer, RegressionTransformerMolecules
)
import mols2grid
from rdkit import Chem
from selfies import encoder
import pandas as pd

# Let us have a look at Buturon, a common herbicide

In [ ]:
smi = "CC(C#C)N(C)C(=O)NC1=CC=C(Cl)C=C1"
Chem.MolFromSmiles(smi)

## Buturon has a water solubility score of -3.90
## We can predict its ESOL (estimated solubility) value with the RegressionTransformer

In [ ]:
config = RegressionTransformerMolecules(search='greedy')
target = f"<esol>[MASK][MASK][MASK][MASK][MASK]|{encoder(smi)}"
esol_predictor = RegressionTransformer(configuration=config, target=target)
score = list(esol_predictor.sample(1))[0]
print(f"For Buturuon, the predicted ESOL is {score}")

#### Ok, we can see that the prediction was decently close but not perfect
#### Now let us try to improve Buturuon to a molecule with higher solubility
#### Note, that we will use the same model to do so!


#### We simply set the hyperparameters for the search and mask undesired parts of the molecule

In [ ]:
config = RegressionTransformerMolecules(search="sample", temperature=2, tolerance=5)
target = "<esol>-3.534|[C][C][Branch1_3][Ring1][C][#C][N][Branch1_3][epsilon][C][C][Branch1_3][epsilon][MASK][MASK][MASK][MASK][C][=C][Branch1_3][epsilon][Cl][C][=C][Ring1][Branch1_2]"

In [ ]:
esol_generator = RegressionTransformer(
    configuration=config, target=target
)
generations = list(esol_generator.sample(5))
generations

In [ ]:
print("\033[1m" "\t\t\tButuruon-inspired molecules with a higher solubility score ")
result = pd.DataFrame({
    "SMILES": [smi] + [s for s,p in generations],
    "Name": ["Buturon"] + 5*["Novel (from RT)"],
    "ESOL": [score] + [round(p, 3) for s,p in generations]
})
mols2grid.display(
    result, tooltip=["Name", "SMILES", "ESOL"],
    fixedBondLength=25, n_cols=6, width="100%", height=None,
    name='Results'
)